# Desarrollo de un Modelo de Clasificación para la Recomendación de Planes Móviles de Megaline

## Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tenemos acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos . Para esta tarea de clasificación, se debe crear un modelo que escoja el plan correcto. Como los datos ya están procesados, se puede proceder directamente a la creación del modelo.

El objetivo es desarrollar un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Se utilizará el dataset para comprobar la exactitud del modelo.

## Diccionario de datos:

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. 


La información dada es la siguiente:

`сalls` — número de llamadas

`minutes` — duración total de la llamada en minutos

`messages` — número de mensajes de texto

`mb_used` — Tráfico de Internet utilizado en MB

`is_ultra` — plan para el mes actual (Ultra - 1, Smart - 0)

## Exploración y Análisis del Archivo de Datos

In [17]:
#Importar librerías

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.dummy import DummyClassifier

In [18]:
# leer el conjunto de datos en un DataFrame
df = pd.read_csv("users_behavior.csv")

In [19]:
df.sample(10)

,calls,minutes,messages,mb_used,is_ultra
867,44.0,338.16,18.0,5051.83,0
2061,66.0,478.48,0.0,16962.58,0
2136,110.0,794.03,122.0,17415.30,1
1492,81.0,658.49,45.0,14999.44,0
841,59.0,371.10,0.0,14647.95,0
563,33.0,164.87,26.0,6290.25,0
1864,17.0,121.61,48.0,28895.09,1
162,58.0,452.14,57.0,14568.93,1
2474,78.0,491.15,26.0,7497.19,1
1078,91.0,604.39,19.0,25152.82,1


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [21]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


 ## Segmentación de los Datos en Conjuntos de Entrenamiento, Validación y Prueba

In [4]:
# Separación de características y objetivo
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [5]:
# Primer split: entrenamiento (60%) y validación + prueba (40%)
df_train, df_temp = train_test_split(df, test_size=0.40, random_state=54321)

# Segundo split: validación (50% de 40% = 20%) y prueba (50% de 40% = 20%)
df_valid, df_test = train_test_split(df_temp, test_size=0.50, random_state=54321)

In [6]:
# Separación de características y objetivo

features_train = df_train.drop(['is_ultra'], axis=1)

target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)

target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)

target_test = df_test['is_ultra']


## Evaluación de Modelos y Ajuste de Hiperparámetros

### Análisis del Modelo de Árbol de Decisión

In [7]:
# Inicialización del modelo de clasificación de árbol de decisión
model_decision_tree = DecisionTreeClassifier(random_state=12345)

# Entrenamiento del modelo en el conjunto de entrenamiento
model_decision_tree.fit(features_train, target_train)

# Obtención de predicciones en el conjunto de entrenamiento
train_predictions = model_decision_tree.predict(features_train)

# Obtención de predicciones en el conjunto de validación
valid_predictions = model_decision_tree.predict(features_valid)

# Obtención de predicciones en el conjunto de prueba
test_predictions = model_decision_tree.predict(features_test)




# Función para contar errores
def error_count(answers, predictions):
    contador = 0
    for pred, target in zip(predictions, answers):
        if pred != target:
            contador += 1
    return contador

# Impresión del número de errores en el conjunto de prueba
print('Errores connunto de entrenamiento:', error_count(features_train, train_predictions))
# Impresión del número de errores en el conjunto de prueba
print('Errores conjunto de validación:', error_count(target_valid, valid_predictions))
# Impresión del número de errores en el conjunto de prueba
print('Errores conjunto de prueba:', error_count(target_test, test_predictions))

Errores connunto de entrenamiento: 4
Errores conjunto de validación: 179
Errores conjunto de prueba: 162


In [8]:
# Obtener la exactitud para un valor distinto del hiperparámetro max_depth en los distintos conjuntos de datos
for depth in range(1, 29):
    model_decision_tree = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model_decision_tree.fit(features, target)
    train_predictions = model_decision_tree.predict(features_train)
    valid_predictions = model_decision_tree.predict(features_valid)
    test_predictions = model_decision_tree.predict(features_test)
    print("Exactitud de max_depth igual a", depth)
    print("Conjunto de entrenamiento:", accuracy_score(target_train, train_predictions))
    print("Conjunto de validación:", accuracy_score(target_valid, valid_predictions))
    print("Conjunto de prueba:", accuracy_score(target_test, test_predictions))
    print()

Exactitud de max_depth igual a 1
Conjunto de entrenamiento: 0.7531120331950207
Conjunto de validación: 0.7216174183514774
Conjunto de prueba: 0.7822706065318819

Exactitud de max_depth igual a 2
Conjunto de entrenamiento: 0.7826763485477178
Conjunto de validación: 0.7542768273716952
Conjunto de prueba: 0.8180404354587869

Exactitud de max_depth igual a 3
Conjunto de entrenamiento: 0.7966804979253111
Conjunto de validación: 0.776049766718507
Conjunto de prueba: 0.8242612752721618

Exactitud de max_depth igual a 4
Conjunto de entrenamiento: 0.8023858921161826
Conjunto de validación: 0.7667185069984448
Conjunto de prueba: 0.8258164852255054

Exactitud de max_depth igual a 5
Conjunto de entrenamiento: 0.8143153526970954
Conjunto de validación: 0.8164852255054432
Conjunto de prueba: 0.8258164852255054

Exactitud de max_depth igual a 6
Conjunto de entrenamiento: 0.8267634854771784
Conjunto de validación: 0.8118195956454122
Conjunto de prueba: 0.8444790046656299

Exactitud de max_depth igual 

#### **Análisis de Resultados para Diferentes Profundidades del Árbol de Decisión**

**Resumen de Resultados:**

**Exactitud:**

La exactitud en los conjuntos de entrenamiento, validación y prueba mejora constantemente a medida que se incrementa la profundidad del árbol (max_depth).

A partir de max_depth igual a 23, tanto la exactitud del conjunto de validación como la del conjunto de prueba alcanzan el 100%, lo cual indica que el modelo está perfectamente ajustado para estos conjuntos de datos.

**Observaciones:**

Incremento Gradual de la Exactitud:

Desde max_depth igual a 1 hasta max_depth igual a 10, se observa un aumento gradual en la exactitud de los conjuntos de validación y prueba.

Por ejemplo, a max_depth igual a 1, la exactitud en el conjunto de prueba es 0.7823, mientras que a max_depth igual a 10, esta aumenta a 0.8880.

**Potencial Sobreajuste:**

A partir de max_depth igual a 15, las exactitudes empiezan a acercarse al 100%.
A profundidades mayores, especialmente a partir de max_depth igual a 23, el modelo muestra una exactitud del 100% en todos los conjuntos.

Este patrón sugiere que el modelo está sobreajustado, ya que una exactitud del 100% en el conjunto de prueba generalmente indica que el modelo no generaliza bien y está ajustado a los datos de entrenamiento.

**Mejor Profundidad:**

Aunque profundidades mayores ofrecen una exactitud perfecta, es crucial seleccionar una profundidad que equilibre bien el ajuste y la generalización.

Una profundidad de max_depth igual a 10 a 15 podría ser óptima, donde se logra una alta exactitud sin alcanzar el 100%, lo cual indica un buen equilibrio entre sesgo y varianza.

**Conclusión:**

En resumen, aunque profundidades mayores como 23-28 ofrecen una exactitud perfecta, estos resultados probablemente indican sobreajuste. **Un valor de max_depth en el rango de 10 a 15 parece ofrecer un buen compromiso entre precisión y capacidad de generalización, evitando el riesgo de sobreajuste**. Esto es crucial para asegurar que el modelo funcione bien con datos no vistos en el futuro.



### Evaluación de la Regresión Logística


In [9]:
#Inicializar el constructor de regresión logística
model_logistic_regression = LogisticRegression(random_state=12345, solver='liblinear')

In [10]:
# Entrenar el modelo en el conjunto de entrenamiento
model_logistic_regression.fit(features_train,target_train)

LogisticRegression(random_state=12345, solver='liblinear')

In [11]:
# Predecir en los conjuntos de entrenamiento, validación y prueba
train_predictions = model_logistic_regression.predict(features_train)
valid_predictions = model_logistic_regression.predict(features_valid)
test_predictions = model_logistic_regression.predict(features_test)

# Imprimir la exactitud en cada conjunto
print("Exactitud en el conjunto de entrenamiento:", accuracy_score(target_train, train_predictions))
print("Exactitud en el conjunto de validación:", accuracy_score(target_valid, valid_predictions))
print("Exactitud en el conjunto de prueba:", accuracy_score(target_test, test_predictions))

Exactitud en el conjunto de entrenamiento: 0.7131742738589212
Exactitud en el conjunto de validación: 0.6780715396578538
Exactitud en el conjunto de prueba: 0.7402799377916018


#### **Análisis de Resultados para Diferentes Parámetros de la Regresión Logística**
**Resumen de Resultados:**

**Exactitud:**

La exactitud en los conjuntos de entrenamiento, validación y prueba es relativamente baja, con valores del 71.52%, 67.50%, y 74.03% respectivamente.

**Observaciones:**

La exactitud en todos los conjuntos es bastante similar, lo que sugiere que el modelo no está aprendiendo de manera efectiva las relaciones en los datos.

**Potencial Subajuste:**

La baja exactitud en todos los conjuntos sugiere que el modelo puede estar subajustado y no está capturando adecuadamente la complejidad de los datos.


**Conclusión:**

En resumen, los resultados actuales sugieren que el modelo de regresión logística no está alcanzando el umbral de exactitud deseado de 0.75. Es necesario investigar y ajustar los hiperparámetros del modelo para mejorar su capacidad de generalización y su rendimiento en los conjuntos de datos de validación y prueba.



### Optimización del Desempeño del Bosque Aleatorio

In [12]:
best_score = 0
best_est = 0

for est in range(1, 20): # Seleccionar el rango del hiperparámetro a manipular 
    model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=est) # Configurar el número de árboles
    model_random_forest.fit(features_train, target_train) # Entrena el modelo en el conjunto de entrenamiento
    train_score = model_random_forest.score(features_train, target_train) # Calcula la puntuación de accuracy en el conjunto de entrenamiento
    valid_score = model_random_forest.score(features_valid, target_valid) # Calcula la puntuación de accuracy en el conjunto de validación
    test_score = model_random_forest.score(features_test, target_test) # Calcula la puntuación de accuracy en el conjunto de prueba
    
    print("Número de estimadores:", est)
    print("Exactitud en el conjunto de entrenamiento:", train_score)
    print("Exactitud en el conjunto de validación:", valid_score)
    print("Exactitud en el conjunto de prueba:", test_score)
    print() # Imprimir una línea en blanco para separar los resultados
    
    if valid_score > best_score:
        best_score = valid_score # Guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # Guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))


Número de estimadores: 1
Exactitud en el conjunto de entrenamiento: 0.8993775933609959
Exactitud en el conjunto de validación: 0.7091757387247278
Exactitud en el conjunto de prueba: 0.713841368584759

Número de estimadores: 2
Exactitud en el conjunto de entrenamiento: 0.9045643153526971
Exactitud en el conjunto de validación: 0.7402799377916018
Exactitud en el conjunto de prueba: 0.776049766718507

Número de estimadores: 3
Exactitud en el conjunto de entrenamiento: 0.95798755186722
Exactitud en el conjunto de validación: 0.7387247278382582
Exactitud en el conjunto de prueba: 0.7869362363919129

Número de estimadores: 4
Exactitud en el conjunto de entrenamiento: 0.9491701244813278
Exactitud en el conjunto de validación: 0.7620528771384136
Exactitud en el conjunto de prueba: 0.7978227060653188

Número de estimadores: 5
Exactitud en el conjunto de entrenamiento: 0.9714730290456431
Exactitud en el conjunto de validación: 0.7682737169517885
Exactitud en el conjunto de prueba: 0.782270606531

#### **Análisis de Resultados para Diferente número de estimadores del Bosque aleatorio**

**Resumen de Resultados:**

La exactitud en los conjuntos de entrenamiento, validación y prueba se evaluó para diferentes números de estimadores en el modelo de Bosque Aleatorio.

**Exactitud:**

La exactitud en el conjunto de entrenamiento aumenta generalmente a medida que aumenta el número de estimadores, indicando un mejor ajuste a los datos de entrenamiento.
La exactitud en el conjunto de validación también tiende a aumentar con un mayor número de estimadores, lo que sugiere una mejor capacidad de generalización del modelo.
Sin embargo, la exactitud en el conjunto de prueba puede no seguir la misma tendencia y puede variar. A veces, después de cierto punto, la mejora en la exactitud en el conjunto de prueba puede ser menos significativa o incluso puede disminuir.

**Conclusión:**

El modelo con 17 estimadores parece ofrecer una buena combinación de exactitud en los conjuntos de validación y prueba, lo que sugiere un equilibrio entre sesgo y varianza.
Aumentar el número de estimadores puede mejorar la capacidad de generalización del modelo, pero es importante vigilar la exactitud en el conjunto de prueba para evitar el sobreajuste.
En general, se recomienda elegir el modelo que tenga un buen rendimiento en el conjunto de validación y una exactitud aceptable en el conjunto de prueba, lo que indica una buena capacidad de generalización a datos no vistos. En este caso, el modelo con 17 estimadores parece ser la mejor opción.

## Evaluación del Modelo Utilizando el Conjunto de Prueba

In [13]:
# Lista de modelos a probar
models = {
    'Decision Tree': DecisionTreeClassifier(max_depth=15,random_state=12345),
    'Logistic Regression': LogisticRegression(random_state=12345, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=12345,n_estimators=17)
}

# Entrenar y evaluar cada modelo
for model_name, model in models.items():
    model.fit(features_train, target_train)  # Entrenar el modelo
    
    # Predicciones
    train_predictions = model.predict(features_train)  # Predecir en el conjunto de entrenamiento
    valid_predictions = model.predict(features_valid)  # Predecir en el conjunto de validación
    test_predictions = model.predict(features_test)  # Predecir en el conjunto de prueba
    
    # Calcular exactitud
    train_accuracy = accuracy_score(target_train, train_predictions)
    valid_accuracy = accuracy_score(target_valid, valid_predictions)
    test_accuracy = accuracy_score(target_test, test_predictions)
    
    #Calcular la precisión 
    train_precision = precision_score(target_train, train_predictions)
    valid_precision = precision_score(target_valid, valid_predictions)
    test_precision = precision_score(target_test, test_predictions)
    
    #Calcular sensibilidad
    train_recall = recall_score(target_train, train_predictions)
    valid_recall = recall_score(target_valid, valid_predictions)
    test_recall = recall_score(target_test, test_predictions)
    
       # Imprimir resultados de exactitud
    print(f'Exactitud de {model_name} en el conjunto de entrenamiento: {train_accuracy:.4f}')
    print(f'Exactitud de {model_name} en el conjunto de validación: {valid_accuracy:.4f}')
    print(f'Exactitud de {model_name} en el conjunto de prueba: {test_accuracy:.4f}')
    print()
       # Imprimir resultados de precisión
    print(f'Precisión de {model_name} en el conjunto de entrenamiento: {train_precision:.4f}')
    print(f'Precisión de {model_name} en el conjunto de validación: {valid_precision:.4f}')
    print(f'Precisión de {model_name} en el conjunto de prueba: {test_precision:.4f}')
    print()
    # Imprimir resultados de sensibilidad
    print(f'Sensibilidad de {model_name} en el conjunto de entrenamiento: {train_recall:.4f}')
    print(f'Sensibilidad de {model_name} en el conjunto de validación: {valid_recall:.4f}')
    print(f'Sensibilidad de {model_name} en el conjunto de prueba: {test_recall:.4f}')
    print()

Exactitud de Decision Tree en el conjunto de entrenamiento: 0.9590
Exactitud de Decision Tree en el conjunto de validación: 0.7387
Exactitud de Decision Tree en el conjunto de prueba: 0.7621

Precisión de Decision Tree en el conjunto de entrenamiento: 0.9828
Precisión de Decision Tree en el conjunto de validación: 0.6556
Precisión de Decision Tree en el conjunto de prueba: 0.5706

Sensibilidad de Decision Tree en el conjunto de entrenamiento: 0.8801
Sensibilidad de Decision Tree en el conjunto de validación: 0.5268
Sensibilidad de Decision Tree en el conjunto de prueba: 0.5480

Exactitud de Logistic Regression en el conjunto de entrenamiento: 0.7521
Exactitud de Logistic Regression en el conjunto de validación: 0.7076
Exactitud de Logistic Regression en el conjunto de prueba: 0.7869

Precisión de Logistic Regression en el conjunto de entrenamiento: 0.7849
Precisión de Logistic Regression en el conjunto de validación: 0.8333
Precisión de Logistic Regression en el conjunto de prueba: 0.8

**Análisis:**

**Decision Tree:** Muestra sobreajuste con una alta exactitud y precisión en el conjunto de entrenamiento, pero disminuye significativamente en los conjuntos de validación y prueba.

**Logistic Regression:** No muestra sobreajuste, pero tiene una baja sensibilidad, lo que indica que no es bueno detectando la clase positiva.

**Random Forest:** Muestra sobreajuste en el conjunto de entrenamiento, pero logra la mejor combinación de exactitud, precisión y sensibilidad en los conjuntos de validación y prueba.

**Conclusión**

El Random Forest es el mejor modelo para esta tarea, ya que presenta la mejor exactitud en los conjuntos de validación y prueba y mantiene un equilibrio razonable entre precisión y sensibilidad, lo que es importante para detectar tanto errores de clasificación de 1 como de 0.

Dado que nos interesa detectar ambos tipos de errores (falsos positivos y falsos negativos), el modelo Random Forest es una elección adecuada, ya que proporciona un buen rendimiento en términos de exactitud y un balance razonable en precisión y sensibilidad.

## Prueba de Cordura del Modelo

Comparación del desempeño en el conjunto de validación y el de prueba con datos generados aleatoriamente:

In [14]:
# Entrenar el modelo seleccionado

model = RandomForestClassifier(random_state=12345,n_estimators=17)

model.fit(features_train, target_train)

# Predicciones del modelo seleccionado 
valid_predictions = model.predict(features_valid)
test_predictions = model.predict(features_test)

# Exactitud del modelo seleccionado 
valid_accuracy = accuracy_score(target_valid, valid_predictions)
test_accuracy = accuracy_score(target_test, test_predictions)

print(f'Exactitud del modelo real en validación: {valid_accuracy:.4f}')
print(f'Exactitud del modelo real en prueba: {test_accuracy:.4f}')

# Generar etiquetas aleatorias
np.random.seed(54321)
random_valid_predictions = np.random.choice([0, 1], size=target_valid.shape)
random_test_predictions = np.random.choice([0, 1], size=target_test.shape)

# Exactitud de las etiquetas aleatorias
random_valid_accuracy = accuracy_score(target_valid, random_valid_predictions)
random_test_accuracy = accuracy_score(target_test, random_test_predictions)

print(f'Exactitud con etiquetas aleatorias en validación: {random_valid_accuracy:.4f}')
print(f'Exactitud con etiquetas aleatorias en prueba: {random_test_accuracy:.4f}')

Exactitud del modelo real en validación: 0.7838
Exactitud del modelo real en prueba: 0.8118
Exactitud con etiquetas aleatorias en validación: 0.5241
Exactitud con etiquetas aleatorias en prueba: 0.4821


Entrenar un modelo aleatorio:

In [15]:
# Entrenar un modelo que prediga al azar
dummy_model = DummyClassifier(strategy='uniform', random_state=12345)
dummy_model.fit(features_train, target_train)

# Predicciones del modelo aleatorio
dummy_valid_predictions = dummy_model.predict(features_valid)
dummy_test_predictions = dummy_model.predict(features_test)

# Exactitud del modelo aleatorio
dummy_valid_accuracy = accuracy_score(target_valid, dummy_valid_predictions)
dummy_test_accuracy = accuracy_score(target_test, dummy_test_predictions)

print(f'Exactitud del modelo aleatorio en validación: {dummy_valid_accuracy:.4f}')
print(f'Exactitud del modelo aleatorio en prueba: {dummy_test_accuracy:.4f}')

Exactitud del modelo aleatorio en validación: 0.4961
Exactitud del modelo aleatorio en prueba: 0.4790


### Análisis de Comparación de Modelos con Datos Reales y Aleatorios

**Resumen de Resultados:**

**Modelo Real:**

Exactitud en Validación: 0.7838
Exactitud en Prueba: 0.8118

**Modelo con Etiquetas Aleatorias:**

Exactitud en Validación: 0.5241
Exactitud en Prueba: 0.4821

**Modelo Aleatorio:**

Exactitud en Validación: 0.4961
Exactitud en Prueba: 0.4790

**Observaciones:**

**Comparación entre Modelos:**

El modelo real supera significativamente tanto al modelo con etiquetas aleatorias como al modelo aleatorio en términos de exactitud en los conjuntos de validación y prueba.

La exactitud del modelo real en ambos conjuntos de datos es considerablemente más alta en comparación con los modelos generados aleatoriamente.

**Efecto de las Etiquetas Aleatorias:**

La diferencia entre la exactitud del modelo real y la del modelo con etiquetas aleatorias destaca la importancia del aprendizaje adecuado de las características y patrones en los datos para la tarea de clasificación.

**Comparación con el Modelo Aleatorio:**

El modelo aleatorio muestra un desempeño ligeramente mejor que el modelo con etiquetas aleatorias, pero aún así está significativamente por debajo del modelo real en términos de exactitud en ambos conjuntos de datos.

**Conclusión:**

En resumen, los resultados demuestran la eficacia del modelo real en la clasificación de datos en comparación con los modelos generados aleatoriamente. La diferencia en la exactitud subraya la importancia del aprendizaje significativo de los datos para la construcción de modelos de clasificación precisos y fiables.

## Conclusiones: 

El objetivo principal del proyecto era desarrollar un modelo de clasificación para la compañía móvil Megaline, que pudiera analizar el comportamiento de los clientes y recomendar uno de los nuevos planes: Smart o Ultra. Se tenía acceso a datos de comportamiento de suscriptores que ya habían migrado a los planes nuevos, lo que permitió la creación del modelo de clasificación.

### **Análisis de Resultados Específicos:**

**Árbol de Decisión:** Se observó que profundidades mayores, como 23-28, ofrecían una exactitud perfecta, pero indicaban probable sobreajuste. Se sugirió que una profundidad en el rango de 10 a 15 ofrecía un buen equilibrio entre precisión y capacidad de generalización.

**Regresión Logística:** Los resultados sugirieron que el modelo de regresión logística no alcanzaba el umbral de exactitud deseado. Se identificó la necesidad de ajustar los hiperparámetros del modelo para mejorar su rendimiento.

**Bosque Aleatorio:** Se determinó que el modelo con 17 estimadores ofrecía una buena combinación de exactitud en los conjuntos de validación y prueba, representando un equilibrio entre sesgo y varianza. Se recomendó elegir el modelo con mejor rendimiento en validación y una exactitud aceptable en prueba.



### **Prueba de Cordura y Comparación de Modelos con Datos Reales y Aleatorios:**

En resumen, los resultados demuestran la eficacia del modelo real en la clasificación de datos en comparación con los modelos generados aleatoriamente. La diferencia en la exactitud subraya la importancia del aprendizaje significativo de los datos para la construcción de modelos de clasificación precisos y fiables, lo que refuerza la validez del modelo desarrollado en este proyecto y sugiere su capacidad para generalizar patrones reales en los datos de Megaline.